In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import h5py

from mpl_toolkits.mplot3d import Axes3D

import sys
sys.path.append("/home/habjan.e/TNG/TNG_cluster_dynamics")
import TNG_DA

### Import particle data

In [ ]:
fName = '/home/habjan.e/TNG/Data/TNG_data/halo_cutouts_dm'

with h5py.File(fName+'.hdf5', 'r') as f:

    part_coordinates = f['PartType1']['Coordinates'][:]
    velocities = f['PartType1']['Velocities'][:]
    ids = f['PartType1']['ParticleIDs'][:]

masses = np.zeros(part_coordinates.shape[0]) + 5.9 * 10**7

coordinates = TNG_DA.coord_cm_corr(cluster_ind = 0, coordinates = part_coordinates)

### Import ROCKSTAR pandas df

In [ ]:
output_path = '/home/habjan.e/TNG/TNG_cluster_dynamics/rockstar/rockstar_subhalos_0_new.list'


column_names = ["id", "pos_0", "pos_1", "pos_2", "pos_3", "pos_4", "pos_5", "bulkvel_0", "bulkvel_1", "bulkvel_2", "corevel_0", "corevel_1", "corevel_2", "mass_grav", "p_start", "num_p"]
df = pd.read_csv(output_path, sep=r"\s+", comment="#", names=column_names)
df

### Add member lists to pandas df

In [ ]:
members = pd.read_csv("rockstar_subhalo_members_0.list",
                      sep=r"\s+", names=["halo_id","particle_id"])
grouped = members.groupby("halo_id")["particle_id"].apply(list)
df["member_ids"] = df["id"].map(grouped)
df.loc[np.where((df['num_p'] > 50))[0]]

### Plot Some subhalos

In [ ]:
fig = plt.figure()
ax  = fig.add_subplot(111, projection='3d')

for i in np.where((df['num_p'] > 50))[0]:

    plot_bool = np.isin(ids, df['member_ids'][i])

    ax.scatter(
        coordinates[plot_bool,0],  # x
        coordinates[plot_bool,1],  # y
        coordinates[plot_bool,2],  # z
        s=2              
    )

# label axes
ax.set_xlabel('X [kpc]')
ax.set_ylabel('Y [kpc]')
ax.set_zlabel('Z [kpc]')

plt.tight_layout()
plt.show()